In [1]:
import pandas as pd
import numpy as np
from math import log2

from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics

def get_x(df):
    return df.loc[:, df.columns != 'class']
    
def get_y(df):
    return df.loc[:, ['class']].T.values[0]

def get_probs_positive_class(pred):
        positive_probs = []

        for prediction in pred:
            positive_probs.append(prediction[1])
        return positive_probs


In [3]:
path = "/home/colombelli/Documents/Experiments16_jul/micarrNPN_Het_borda/fold_1/"
th = "agg_ranking_th150.csv"

rank = pd.read_csv(path+th, index_col=0)
rank

,rank
PLP1,1692
FKBPL,2037
PRCC,2506
ODF2,3055
ENTPD7,3130
...,...
TMEM50A,83451
THNSL2,83769
ZNF644,84546
TRO,85040


In [4]:
mdf = pd.read_csv('/home/colombelli/Documents/datasets/assembler/brca_micarr_npn.csv', index_col=0)
mdf

,X15E1.2,X2..PDE,X7A5,A1BG,A2BP1,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,...,ZW10,ZWINT,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,class
TCGA-A1-A0SD-01A-11R-A115-07,-0.025539,-0.515257,1.145865,1.326113,2.067113,-0.539656,-0.199191,-1.106178,0.457877,0.063882,...,-1.606837,0.231849,2.283435,-2.067113,-0.524980,-0.811783,-0.897097,-0.982508,0.666956,1
TCGA-A1-A0SE-01A-11R-A084-07,-1.845034,0.754251,-0.594494,0.145203,-0.223118,0.324723,-0.306843,1.038760,0.184023,-0.244979,...,-0.709938,0.042572,0.110878,-0.271366,-0.333701,-0.203532,-0.140904,-1.562952,0.034054,1
TCGA-A1-A0SH-01A-11R-A084-07,-0.962187,-0.630318,0.682929,1.053336,0.262550,0.443767,-0.356263,1.083167,0.320244,0.989371,...,-1.222157,-0.743039,0.748633,-0.955499,-0.982508,0.515257,1.162246,-0.063882,-0.149505,1
TCGA-A1-A0SJ-01A-11R-A084-07,-1.296385,1.622164,0.197021,1.060707,0.149505,0.865918,0.029796,0.034054,1.249202,0.853676,...,-1.038760,-0.300164,2.492269,-0.989371,-0.656400,0.392743,-0.853676,-0.765557,-0.975690,1
TCGA-A1-A0SK-01A-12R-A084-07,0.227481,-0.996281,-1.622164,-1.003237,-1.622164,-1.741719,-0.498357,1.346590,-1.917733,-0.897097,...,2.732881,1.316076,0.115161,0.743039,0.406554,1.031553,-2.732881,-0.429744,0.534752,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-E2-A1B5-01A-21R-A12P-07,0.379005,-0.549504,-0.731918,-0.347217,0.794253,0.539656,1.170553,-0.968917,-0.220938,0.034054,...,0.425088,0.844577,-1.378389,1.162246,1.024398,-2.283435,-0.102319,1.231074,-0.500762,1
TCGA-E2-A1B6-01A-31R-A12P-07,-0.128022,0.240598,0.495954,0.955499,-1.017295,0.329208,0.661669,-2.067113,1.741719,-1.577250,...,-2.142156,-1.162246,0.171056,-0.757068,0.805913,-2.067113,-0.609753,2.103180,-0.811783,1
TCGA-E2-A1BC-01A-11R-A12P-07,-1.129780,0.776960,-0.210052,-0.609753,-0.903436,0.042572,-1.637875,2.103180,0.008512,0.709938,...,0.223118,-1.154017,-0.909811,-0.324723,-0.153809,0.955499,0.342705,-1.068135,-1.204604,1
TCGA-E2-A1BC-11A-32R-A12P-07,-0.351737,0.491156,-1.508684,-2.410722,-2.033510,1.306168,0.441424,0.693673,1.434655,0.369886,...,0.042572,-2.410722,0.320244,-1.068135,-2.957141,1.098444,0.599565,0.929158,-0.847602,0


In [5]:
rdf = pd.read_csv('/home/colombelli/Documents/datasets/assembler/brca_rna_npn.csv', index_col=0)
rdf

,A1BG,A1CF,A2BP1,A2LD1,A2M,A2ML1,A4GALT,A4GNT,AAA1,AAAS,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,class
TCGA-3C-AAAU-01A-11R-A41B-07,0.514547,-0.239382,-0.663427,0.486411,-1.124216,-0.538305,-1.727564,2.655164,1.541330,-0.166443,...,0.505125,2.104695,1.471254,1.755471,1.407684,0.852087,-0.062171,1.090010,0.680262,1
TCGA-3C-AALI-01A-11R-A41B-07,0.811243,-0.239382,-0.663427,-0.381783,-0.677659,0.060097,-0.693342,0.103737,-0.118323,0.906769,...,0.555116,-0.204364,-0.352997,0.885193,1.060632,-1.025138,0.925602,0.193804,-1.826446,1
TCGA-3C-AALJ-01A-31R-A41B-07,1.489653,1.805237,-0.663427,1.319284,-0.444903,-1.727564,1.046278,-0.972519,-0.118323,0.236187,...,2.163996,-1.135912,0.060097,-1.407684,1.765082,-1.339160,0.909886,-0.613994,-1.396714,1
TCGA-3C-AALK-01A-11R-A41B-07,0.467864,-0.239382,-0.663427,-0.664715,-0.145481,-0.424434,0.837341,-0.972519,-0.118323,0.521644,...,0.014498,-0.375113,-0.428967,-1.018191,0.885193,-1.018191,1.018191,-1.001034,-0.634088,1
TCGA-4H-AAAK-01A-12R-A41B-07,0.977494,1.359574,1.477332,1.271727,-0.342003,-0.066322,0.272544,-0.045579,1.692311,0.651880,...,0.022784,-0.569650,-0.514547,-0.234059,-1.258003,-0.328865,0.204364,-0.858036,-0.070474,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-WT-AB44-01A-11R-A41B-07,1.612274,-0.239382,-0.663427,-0.690717,-1.188475,0.231931,0.304920,0.368460,-0.118323,2.004285,...,-1.334142,-1.675438,-1.612274,-1.385909,0.283303,-2.283679,2.342268,-0.957735,-1.659026,1
TCGA-XX-A899-01A-11R-A36F-07,0.706542,-0.239382,0.259674,1.025138,0.701248,-1.131997,0.828580,0.348594,-0.118323,0.132935,...,-0.477117,0.198025,0.559948,0.609009,-1.025138,-0.703892,0.755113,1.159778,-0.746898,1
TCGA-XX-A89A-01A-11R-A36F-07,0.913011,2.448625,0.977494,-0.097493,0.484084,0.819881,1.151750,1.534645,2.208168,0.967568,...,0.074627,-0.586759,-0.928773,1.007862,0.828580,-0.451766,0.919289,0.406387,0.128757,1
TCGA-Z7-A8R5-01A-42R-A41B-07,1.534645,-0.239382,0.477117,-0.060097,1.521478,-0.244712,0.662139,-0.972519,-0.118323,1.413232,...,-1.035650,-1.299911,-1.582945,-1.299911,-0.324498,-1.435868,1.339160,-1.067891,-0.470174,1


In [7]:
donthave = []
have = []
genes = list(rank[0:150].index)
for gene in genes:
    try:
        rdf[gene]
        have.append(gene)
    except:
        donthave.append(gene)

In [8]:
len(have)

141

In [9]:
len(donthave)

9

In [10]:
df = mdf[have + ['class']]
df

,PLP1,FKBPL,PRCC,ODF2,ENTPD7,NUDT3,HIST1H3H,FAM70A,CAV3,PACSIN1,...,GAS2,TNFRSF4,CORO2B,FSTL4,TLE3,CST1,CXCL2,EPHA2,PID1,class
TCGA-A1-A0SD-01A-11R-A115-07,-0.884524,-1.010241,-1.068135,-0.841559,-0.467333,-0.635500,0.847602,-0.360797,0.699074,-1.170553,...,1.121844,-0.604651,-0.759892,0.434409,0.253755,-0.093767,-0.651149,-1.653994,1.249202,1
TCGA-A1-A0SE-01A-11R-A084-07,0.495954,-0.324723,0.955499,0.280202,0.450811,0.158117,0.231849,0.614870,0.076681,0.534752,...,0.166740,-0.929158,0.515257,-0.078816,0.284628,-0.188352,0.347217,-0.093767,0.599565,1
TCGA-A1-A0SH-01A-11R-A084-07,0.258150,-0.392743,-0.614870,0.055356,1.346590,0.581882,-0.996281,1.577250,0.038313,0.128022,...,0.063882,-0.666956,0.829562,-1.121844,-0.338199,0.258150,0.034054,-0.365337,1.521841,1
TCGA-A1-A0SJ-01A-11R-A084-07,0.574358,-1.003237,-0.500762,-1.053336,-1.031553,-1.548961,-0.379005,-0.038313,0.726391,-1.286722,...,0.569360,-1.495779,0.214403,0.884524,-0.429744,0.347217,-0.342705,0.534752,0.429744,1
TCGA-A1-A0SK-01A-12R-A084-07,2.594339,-0.038313,0.179698,1.470679,-0.715401,-0.630318,0.102319,-0.859782,-1.723111,2.410722,...,0.835546,-0.554448,0.136608,0.038313,-0.748633,1.357040,-0.374442,-1.458464,0.415803,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-E2-A1B5-01A-21R-A12P-07,0.527418,-1.346590,0.476830,0.029796,-1.053336,2.410722,1.129780,-0.379005,-0.025539,1.705072,...,0.448460,2.492269,0.922671,-0.903436,-0.342705,-0.510414,0.262550,-0.266956,-0.017025,1
TCGA-E2-A1B6-01A-31R-A12P-07,0.284628,0.397338,-0.495954,1.267742,-0.486370,-0.919442,-0.392743,0.709938,0.829562,0.776960,...,0.884524,2.184621,0.599565,0.197021,0.149505,0.737467,1.046021,0.457877,0.392743,1
TCGA-E2-A1BC-01A-11R-A12P-07,-0.457877,0.823607,-1.357040,-0.495954,0.975690,-0.734690,0.811783,0.057487,0.443767,-0.240598,...,-0.520113,-0.534752,0.524980,-0.320244,-0.609753,0.293497,0.935683,0.190518,0.645916,1
TCGA-E2-A1BC-11A-32R-A12P-07,1.357040,-1.495779,-1.458464,-1.326113,-2.410722,-1.187410,-2.342426,1.137786,2.594339,-1.106178,...,-0.374442,0.680255,2.283435,-2.050027,-0.922671,-0.765557,1.868200,0.171056,2.142156,0


In [11]:
tedf = rdf[have + ['class']]
tedf

,PLP1,FKBPL,PRCC,ODF2,ENTPD7,NUDT3,HIST1H3H,FAM70A,CAV3,PACSIN1,...,GAS2,TNFRSF4,CORO2B,FSTL4,TLE3,CST1,CXCL2,EPHA2,PID1,class
TCGA-3C-AAAU-01A-11R-A41B-07,-0.659568,0.105819,0.938340,0.366245,0.291934,-0.210711,2.283679,-0.460951,-0.557531,0.837341,...,0.406387,0.074627,-0.479437,1.643044,0.467864,1.135912,0.193804,-1.495899,-0.344199,1
TCGA-3C-AALI-01A-11R-A41B-07,-0.900562,1.060632,0.397413,2.953119,-0.752369,-1.447464,2.086462,-0.616492,2.068891,1.521478,...,-0.864016,1.184317,-0.037288,0.706542,-0.840276,0.093332,-1.039180,-0.124582,-0.217067,1
TCGA-3C-AALJ-01A-31R-A41B-07,-1.112671,-0.808377,0.274693,2.283679,-0.616492,-1.028629,1.071542,-1.319284,-0.557531,0.524015,...,-0.370676,1.453336,-0.474800,-0.876069,-0.166443,-0.897471,-0.491073,-1.067891,-0.659568,1
TCGA-3C-AALK-01A-11R-A41B-07,0.198025,0.014498,0.562369,0.601560,0.261816,0.066322,-0.377335,0.055947,0.559948,0.002071,...,-0.114153,1.018191,0.463253,-0.714520,0.552705,-0.500430,0.035215,0.526390,0.438060,1
TCGA-4H-AAAK-01A-12R-A41B-07,-0.408635,0.122495,-0.185371,0.964280,0.377335,-1.049846,-1.101271,1.053428,0.574521,-0.158050,...,-0.586759,0.609009,0.270396,0.479437,-0.302753,0.126669,-0.613994,-0.147574,0.431237,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-WT-AB44-01A-11R-A41B-07,-1.147764,1.324204,1.380566,2.004285,-0.611500,1.508566,-0.601560,-0.509830,-0.557531,0.417651,...,-0.328865,0.463253,0.876069,-0.302753,-0.495746,1.014736,0.858036,-0.649326,-0.463253,1
TCGA-XX-A899-01A-11R-A36F-07,0.062171,-1.011293,-0.555116,-0.193804,-0.664715,-0.990879,-1.163820,0.227679,1.541330,0.831493,...,0.322316,1.112671,1.612274,-0.166443,-0.276844,0.041433,0.799819,0.780070,0.894389,1
TCGA-XX-A89A-01A-11R-A36F-07,0.538305,-0.124582,-0.395174,0.397413,0.888250,-0.064247,-0.885193,-0.158050,0.626524,-0.755113,...,0.272544,1.188475,1.396714,0.422171,0.335427,0.524015,0.533530,-0.404140,1.035650,1
TCGA-Z7-A8R5-01A-42R-A41B-07,-0.145481,1.534645,0.609009,-0.101655,-1.643044,0.774483,-0.502776,0.805518,0.741448,-0.672468,...,1.035650,0.873044,0.072550,0.444903,-0.514547,-0.221309,1.304709,0.328865,-0.257533,1


In [12]:
clf = SVC(gamma='auto', probability=True)

X_train = get_x(df)
y_train = get_y(df)

X_test = get_x(tedf)
y_test = get_y(tedf)


clf.fit(X_train, y_train)
clf.score(X_test, y_test)
#np.argmax(clf.predict_proba(X_test), axis=1)

0.9950617283950617

In [16]:
pred = clf.predict_proba(X_test)
pred = get_probs_positive_class(pred)

metrics.roc_auc_score(np.array(y_test, dtype=int), pred)

0.999140741692498

### Now training with RNA-Seq and testing in Microarray

In [17]:
path = "/home/colombelli/Documents/Experiments16_jul/rnaseqNPN_Het_borda/fold_1/"
th = "agg_ranking_th150.csv"

rank = pd.read_csv(path+th, index_col=0)
rank

,rank
CCL11,1093
LOC283867,1132
AIM1L,1680
PPAPDC1A,1699
EPYC,1728
...,...
TTTY7,100135
XKRY,100283
VTRNA1.1,100286
VTRNA1.2,100290


In [18]:
donthave = []
have = []
genes = list(rank[0:150].index)
for gene in genes:
    try:
        mdf[gene]
        have.append(gene)
    except:
        donthave.append(gene)

In [19]:
len(have)

128

In [20]:
len(donthave)

22

In [21]:
donthave

['LOC283867',
 'GLRA4',
 'LOC284578',
 'P2RX6',
 'C3orf50',
 'LOC388387',
 'MESTIT1',
 'GPR182',
 'C2orf71',
 'C13orf36',
 'FLJ42875',
 'DEFB132',
 'C11orf87',
 'LOC121838',
 'FAM183A',
 'GBAP1',
 'LOC642597',
 'ACCS',
 'FLJ13197',
 'ARAP3',
 'LOC158696',
 'PCDHGA11']

In [22]:
df = rdf[have + ['class']]
df

,CCL11,AIM1L,PPAPDC1A,EPYC,SLC24A2,MMP13,DACT2,DCT,KCTD8,PRRG3,...,HBA2,SLC6A16,RAB25,SCT,AOX1,ARL4A,PPYR1,LASS2,LCN6,class
TCGA-3C-AAAU-01A-11R-A41B-07,-1.001034,1.049846,-0.484084,-0.509830,-1.039180,-0.500430,-0.805518,-0.070474,-0.852087,-1.253480,...,0.662139,-0.049726,0.604039,1.369995,-0.828580,-1.209583,-0.564793,1.907914,0.234059,1
TCGA-3C-AALI-01A-11R-A41B-07,-0.780070,1.736718,0.693342,1.895492,-0.422171,0.189585,-0.033143,-0.853571,-0.852087,0.028999,...,0.251117,-0.217067,-0.361822,1.489653,-0.954478,-0.521644,-0.291934,-0.135024,-0.862519,1
TCGA-3C-AALJ-01A-31R-A41B-07,0.557531,0.701248,0.984165,0.822774,-0.311433,0.837341,-1.692311,-0.853571,-0.852087,-1.253480,...,-0.070474,-1.413232,0.442619,1.101271,-0.944766,0.675061,-1.233435,0.208595,-0.862519,1
TCGA-3C-AALK-01A-11R-A41B-07,1.039180,0.294095,1.180178,0.120409,1.090010,1.105055,0.101655,0.017605,0.112069,0.897471,...,0.204364,0.967568,0.782873,0.607765,-0.076704,0.498087,-1.233435,0.244712,0.026927,1
TCGA-4H-AAAK-01A-12R-A41B-07,0.763380,0.091252,1.709674,0.629041,2.123649,0.488741,-0.085015,0.744170,0.574521,-0.488741,...,-0.263959,-1.192655,-0.364033,1.837363,-0.105819,0.024855,-0.066322,-0.831493,-0.862519,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-WT-AB44-01A-11R-A41B-07,2.231959,-0.255393,2.538598,2.185554,-0.479437,1.659026,-0.035215,0.711856,-0.852087,-0.130846,...,0.891315,-0.183264,1.765082,2.538598,-0.283303,0.916146,0.331050,1.028629,0.285459,1
TCGA-XX-A899-01A-11R-A36F-07,0.210711,1.042722,0.538305,-0.014498,0.474800,0.922441,0.486411,0.819881,1.568773,0.808377,...,1.042722,0.771698,-0.693342,0.682868,0.984165,-0.031071,0.435783,-0.604039,0.454059,1
TCGA-XX-A89A-01A-11R-A36F-07,0.987516,1.180178,0.606522,2.123649,0.644230,1.244510,0.626524,0.074627,-0.852087,0.227679,...,0.967568,0.128757,-1.582945,1.124216,1.344212,0.272544,-0.010355,-0.791318,0.599084,1
TCGA-Z7-A8R5-01A-42R-A41B-07,2.086462,-0.114153,0.074627,-0.688096,-0.253255,-0.451766,0.811243,0.547893,1.521478,-0.296257,...,-0.611500,0.422171,0.559948,1.528029,0.755113,2.374835,0.557531,0.202250,2.208168,1


In [23]:
tedf = mdf[have + ['class']]
tedf

,CCL11,AIM1L,PPAPDC1A,EPYC,SLC24A2,MMP13,DACT2,DCT,KCTD8,PRRG3,...,HBA2,SLC6A16,RAB25,SCT,AOX1,ARL4A,PPYR1,LASS2,LCN6,class
TCGA-A1-A0SD-01A-11R-A115-07,0.171056,-0.584397,1.845034,1.357040,0.472076,1.917733,-0.320244,-0.181860,-0.594494,-1.446459,...,-1.068135,-0.448460,-1.178940,0.324723,0.282415,0.481595,-0.491156,-0.554448,-0.635500,1
TCGA-A1-A0SE-01A-11R-A084-07,0.104458,-0.266956,0.515257,-0.231849,0.338199,0.093767,1.154017,0.335949,0.153809,0.909811,...,-0.231849,0.635500,1.336282,-0.044702,-0.559405,0.365337,-0.068148,1.267742,0.429744,1
TCGA-A1-A0SH-01A-11R-A084-07,1.249202,-0.175375,2.594339,1.845034,2.594339,1.046021,0.640700,0.559405,0.397338,1.075621,...,-0.693673,-0.897097,-1.106178,1.705072,0.236221,0.306843,0.333701,-0.476830,-1.083167,1
TCGA-A1-A0SJ-01A-11R-A084-07,-0.737467,-0.853676,-0.443767,-0.666956,-0.800070,0.942248,0.306843,0.089493,1.357040,0.034054,...,1.622164,-0.404247,-1.046021,-0.975690,1.446459,0.175375,-0.038313,1.316076,1.411620,1
TCGA-A1-A0SK-01A-12R-A084-07,-0.420441,1.367638,-0.315771,-0.962187,-0.264752,-1.195964,1.944342,-1.892396,0.401942,2.732881,...,-2.957141,0.034054,-1.240087,-1.010241,-2.231354,-1.075621,-1.389299,-1.687562,-1.098444,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-E2-A1B5-01A-21R-A12P-07,0.266956,-0.102319,-0.942248,-1.024398,-0.374442,-1.145865,0.841559,1.098444,0.262550,-0.569360,...,0.500762,1.357040,-1.162246,1.306168,-0.640700,-0.329208,0.989371,-1.378389,-0.481595,1
TCGA-E2-A1B6-01A-31R-A12P-07,2.184621,0.584397,-0.909811,-0.672262,0.709938,-1.591872,0.476830,0.038313,-1.222157,0.136608,...,0.704496,0.620003,-0.258150,-0.093767,0.574358,0.666956,1.162246,0.376722,1.780823,1
TCGA-E2-A1BC-01A-11R-A12P-07,-0.524980,-0.218758,0.800070,0.275781,2.103180,0.589438,-1.154017,-1.687562,0.771246,-0.158117,...,1.367638,-0.510414,-0.280202,1.495779,0.262550,-0.897097,-1.326113,-0.046833,1.423045,1
TCGA-E2-A1BC-11A-32R-A12P-07,-1.760940,-2.732881,-2.283435,-0.320244,0.962187,-0.916223,1.446459,-0.034054,2.594339,2.184621,...,2.231354,1.411620,-2.594339,-1.027969,1.822802,0.275781,1.267742,-1.760940,1.622164,0


In [24]:
clf = SVC(gamma='auto', probability=True)

X_train = get_x(df)
y_train = get_y(df)

X_test = get_x(tedf)
y_test = get_y(tedf)


clf.fit(X_train, y_train)
clf.score(X_test, y_test)
#np.argmax(clf.predict_proba(X_test), axis=1)

0.9831365935919055

In [25]:
pred = clf.predict_proba(X_test)
pred = get_probs_positive_class(pred)

metrics.roc_auc_score(np.array(y_test, dtype=int), pred)

1.0

In [26]:
list(np.argmax(clf.predict_proba(X_test), axis=1))

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
